In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_selection import f_regression, mutual_info_regression

In [2]:
all_cars = pd.read_csv('dataset_V2_1.csv')
all_cars.head(3)

,Unnamed: 0,bodyType,brand,car_url,color,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price
0,0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,Машина в отличном состоянии🥇\nКомплектация Ele...,1.8 LTR,152 N12,"{'cruise-control': True, 'tinted-glass': True,...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,2.0,DUPLICATE,FORWARD_CONTROL,LEFT,True,True,829000.0
1,1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,бежевый,Хорошее техническое и косметическое\nсостояние...,1.8 LTR,152 N12,"{'cruise-control': True, 'esp': True, 'adaptiv...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,3.0,ORIGINAL,FORWARD_CONTROL,LEFT,True,True,785000.0
2,2,внедорожник 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/yeti/1105...,пурпурный,Идеальное состояние автомобиля\nДилер Краснояр...,1.2 LTR,105 N12,"{'computer': True, 'ptf': True, 'rain-sensor':...",бензин,...,ALLROAD_5_DOORS ROBOT 1.2,роботизированная,EUROPEAN,1.0,ORIGINAL,FORWARD_CONTROL,LEFT,True,True,745000.0


In [3]:
all_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104663 entries, 0 to 104662
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            104663 non-null  int64  
 1   bodyType              104450 non-null  object 
 2   brand                 104451 non-null  object 
 3   car_url               104663 non-null  object 
 4   color                 104451 non-null  object 
 5   description           104451 non-null  object 
 6   engineDisplacement    104451 non-null  object 
 7   enginePower           104451 non-null  object 
 8   equipment_dict        104449 non-null  object 
 9   fuelType              104449 non-null  object 
 10  mileage               104449 non-null  float64
 11  modelDate             104450 non-null  float64
 12  model_info            97958 non-null   object 
 13  model_name            104449 non-null  object 
 14  numberOfDoors         104450 non-null  float64
 15  

датасет доставточно "грязный", проверим на наличие пропусков, уберем лишние данные и преобразуем имеющиеся в формат, пригодный для анализа

In [4]:
all_cars.isnull().sum()

Unnamed: 0                 0
bodyType                 213
brand                    212
car_url                    0
color                    212
description              212
engineDisplacement       212
enginePower              212
equipment_dict           214
fuelType                 214
mileage                  214
modelDate                213
model_info              6705
model_name               214
numberOfDoors            213
productionDate           212
super_gen                215
vehicleConfiguration     213
vehicleTransmission      214
vendor                  6704
Владельцы                221
ПТС                      216
Привод                   216
Руль                     214
Состояние                233
Таможня                  223
price                   8750
dtype: int64

пропуски есть практически в каждой колонке, но их сравнительно незначительнное количетсво. Поэтому спокойно их удаляем

In [5]:
all_cars = all_cars.dropna()
all_cars.isnull().sum()

Unnamed: 0              0
bodyType                0
brand                   0
car_url                 0
color                   0
description             0
engineDisplacement      0
enginePower             0
equipment_dict          0
fuelType                0
mileage                 0
modelDate               0
model_info              0
model_name              0
numberOfDoors           0
productionDate          0
super_gen               0
vehicleConfiguration    0
vehicleTransmission     0
vendor                  0
Владельцы               0
ПТС                     0
Привод                  0
Руль                    0
Состояние               0
Таможня                 0
price                   0
dtype: int64

In [9]:
# признак 'car_url' будем ипользовать в качестве идентификатора, извдечем оттуда id объявления
all_cars['car_url'][5]

'https://auto.ru/cars/used/sale/skoda/octavia/1105846102-35605d51/'

In [6]:
all_cars.url_split = all_cars.car_url.str.split('/')
all_cars.url_split[15648]

<ipython-input-6-a395f9173830>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  all_cars.url_split = all_cars.car_url.str.split('/')


['https:',
 '',
 'auto.ru',
 'cars',
 'used',
 'sale',
 'honda',
 'accord',
 '1105508916-b03d1806',
 '']

In [7]:
id_car = [power[0:][8] for power in all_cars.url_split]
id_car_split = [x.split('-') for x in id_car]
all_cars['id_car'] = [q[0] for q in id_car_split]
all_cars['id_car'] = pd.to_numeric(all_cars['id_car'])
all_cars['id_car']

0         1105765221
1         1105672391
2         1105201873
3         1105607078
4         1105561522
             ...    
104656    1105731924
104658    1104571613
104659    1105831300
104660    1105408230
104661    1105396628
Name: id_car, Length: 95876, dtype: int64

In [8]:
all_cars['id_car'].duplicated().value_counts()

False    95114
True       762
Name: id_car, dtype: int64

в id объявлений встречаются дубликаты, но их небольшое количетсво, удаляем их

In [9]:
all_cars = all_cars.drop_duplicates(subset='id_car', keep=False, inplace=False)

рассмотрим признак super_gen, где собраны характеристики автомобиля. бОльшая часть дублируется в других признаках, но здесь есть информация о расходе топлива, что может быть полезной для построения модели. Выделим его в отдельный признак 

In [43]:
all_cars[['id_car', 'super_gen']].loc[12][1]

"{'id': '5040501', 'name': 'Scout', 'nameplate': 'Scout', 'displacement': 1798, 'engine_type': 'GASOLINE', 'gear_type': 'ALL_WHEEL_DRIVE', 'transmission': 'MECHANICAL', 'power': 152, 'power_kvt': 112, 'human_name': 'Scout 1.8 MT (152 л.с.) 4WD', 'acceleration': 8.7, 'clearance_min': 179, 'fuel_rate': 7.8}"

In [10]:
all_cars['rate_split'] = all_cars['super_gen'].str.split(',')
all_cars['rate_split'][38]

["{'id': '6038953'",
 " 'displacement': 1798",
 " 'engine_type': 'GASOLINE'",
 " 'gear_type': 'ALL_WHEEL_DRIVE'",
 " 'transmission': 'MECHANICAL'",
 " 'power': 152",
 " 'power_kvt': 112",
 " 'human_name': '1.8 MT (152 л.с.) 4WD'",
 " 'acceleration': 8.7",
 " 'clearance_min': 180",
 " 'fuel_rate': 8}"]

как видно, есть данные, которые попали в список случайно. обработаем их

In [11]:
# создадим список, куда могут попасть все данные о расходе топлива
rate = []
for d in all_cars['rate_split']:
    rate.append(d[-2:])  
rate[50:55]

[[" 'clearance_min': 170", " 'fuel_rate': 7.7}"],
 [" 'clearance_min': 164", " 'fuel_rate': 7.4}"],
 [" 'clearance_min': 180", " 'fuel_rate': 6.4}"],
 [" 'clearance_min': 149", " 'fuel_rate': 6.9}"],
 [" 'clearance_min': 149", " 'fuel_rate': 7.5}"]]

In [12]:
# извлечем из него только данные о расходе
rate2 = []
for fuel in rate:
    if "fuel_rate" in fuel[0]:
            rate2.append(fuel[0])
    elif "fuel_rate" in fuel[1]:
            rate2.append(fuel[1])
    elif "fuel_rate" not in fuel:
            rate2.append('w') 
            
all_cars['rate'] = rate2
all_cars['rate']

0           'fuel_rate': 7.7}
1           'fuel_rate': 7.7}
2           'fuel_rate': 6.4}
3           'fuel_rate': 6.3}
4           'fuel_rate': 7.4}
                 ...         
104656      'fuel_rate': 6.8}
104658     'fuel_rate': 14.4}
104659      'fuel_rate': 7.8}
104660      'fuel_rate': 9.5}
104661                      w
Name: rate, Length: 94352, dtype: object

In [13]:
# "почистим" список и разложим его на 2 части
all_cars['rate'] = all_cars['rate'].str.replace(('}'), '')
all_cars['rate'] = all_cars['rate'].str.replace(('"'), '')
all_cars['rate_split'] = all_cars['rate'].str.split(':')

In [14]:
rate3 = []
for fuel in all_cars['rate_split']:
    if len(fuel)> 1:
        rate3.append(fuel[1])
    else:
        rate3.append(fuel[0])
        
all_cars['rate2'] = rate3
all_cars['rate2'].value_counts()

w        5216
 7.4     2437
 6.7     2293
 7.5     2208
 7.9     2202
         ... 
 19.2       1
 17.3       1
 17.8       1
 1.4        1
 19.4       1
Name: rate2, Length: 164, dtype: int64

сначала найдем медиану и среднее списка не учитывая пропуски

In [15]:
rate_w = all_cars[~all_cars.rate2.str.match("w", na=False)]
rate_w['rate2'] = pd.to_numeric(rate_w['rate2'])

<ipython-input-15-31dfc8d30bfc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate_w['rate2'] = pd.to_numeric(rate_w['rate2'])


In [16]:
print (rate_w['rate2'].median())
print(rate_w['rate2'].mean())
print (rate_w['rate2'].mode())

7.8
8.198937578531964
0    7.4
dtype: float64


In [17]:
# заменим пропуски, которые мы ранее обозначили через 'w' медианой признака, и преобразуем данные в числовой формат
all_cars['rate2'] = all_cars['rate2'].str.replace(('w'), '7.8')
all_cars['rate_fuel'] = pd.to_numeric(all_cars['rate2'])
all_cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94352 entries, 0 to 104661
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            94352 non-null  int64  
 1   bodyType              94352 non-null  object 
 2   brand                 94352 non-null  object 
 3   car_url               94352 non-null  object 
 4   color                 94352 non-null  object 
 5   description           94352 non-null  object 
 6   engineDisplacement    94352 non-null  object 
 7   enginePower           94352 non-null  object 
 8   equipment_dict        94352 non-null  object 
 9   fuelType              94352 non-null  object 
 10  mileage               94352 non-null  float64
 11  modelDate             94352 non-null  float64
 12  model_info            94352 non-null  object 
 13  model_name            94352 non-null  object 
 14  numberOfDoors         94352 non-null  float64
 15  productionDate    

In [18]:
# удалим лишние признаки
all_cars.drop(['rate_split', 'rate2', 'rate'], axis=1, inplace=True)
all_cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94352 entries, 0 to 104661
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            94352 non-null  int64  
 1   bodyType              94352 non-null  object 
 2   brand                 94352 non-null  object 
 3   car_url               94352 non-null  object 
 4   color                 94352 non-null  object 
 5   description           94352 non-null  object 
 6   engineDisplacement    94352 non-null  object 
 7   enginePower           94352 non-null  object 
 8   equipment_dict        94352 non-null  object 
 9   fuelType              94352 non-null  object 
 10  mileage               94352 non-null  float64
 11  modelDate             94352 non-null  float64
 12  model_info            94352 non-null  object 
 13  model_name            94352 non-null  object 
 14  numberOfDoors         94352 non-null  float64
 15  productionDate    

In [19]:
print(all_cars.model_name.unique())
len(all_cars.model_name.unique())

['Octavia' 'Yeti' 'Fabia' 'Roomster' 'Octavia RS' 'Superb' 'Fabia RS'
 'Rapid' 'Kodiaq' 'Karoq' 'Felicia' 'A6' 'A4' 'A8' 'Q7' 'Q5' 'R8' 'A5'
 'RS 3' 'Q3' 'A7' 'RS 5' 'A3' 'A1' 'A4 allroad' 'A6 allroad' 'S5' 'S4'
 'S3' 'TT' 'S7' 'S6' 'S8' 'RS Q3' 'TT RS' 'SQ5' 'RS 6' 'RS 7' 'TTS' 'RS 4'
 'SQ7' 'Q8' 'e-tron' '80' '100' '90' 'A2' 'V8' 'Coupe' '200'
 'Q3 Sportback' 'SQ8' 'e-tron Sportback' '920' 'Accord' 'CR-V' 'Civic'
 'Civic Type R' 'Jazz' 'Freed' 'Pilot' 'Stream' 'Odyssey' 'Crossroad'
 'Fit' 'Mobilio Spike' 'Stepwgn' 'Elysion' 'Legend' 'Ridgeline' 'S2000'
 'Airwave' 'Partner' 'Life' 'City' 'Fit Aria' 'Zest' 'Mobilio' 'Logo'
 'Integra' 'Orthia' 'Domani' 'HR-V' 'Civic Ferio' 'Torneo' 'Capa'
 'Integra SJ' 'S-MX' 'Inspire' 'Prelude' 'Passport' 'Shuttle'
 'Fit Shuttle' 'Crosstour' 'CR-Z' 'Insight' 'N-BOX' 'FR-V' 'Edix'
 'Element' 'Vamos' 'Ascot' 'CR-X' 'Saber' 'Avancier' "That'S"
 'Odyssey (North America)' 'Acty' 'Z' 'MDX' 'Vezel' 'N-WGN' 'Rafaga'
 'Beat' 'Ascot Innova' 'Vigor' 'Clarity' 'Ja

633

признак модели достаточно обширен (более 600), но при это одни и те же модели авто встречаются в разном написании. Почистим его

In [20]:
def clean_model(data):
    data['model_name'] = data['model_name'].str.strip()
    data['model_name'] = data['model_name'].str.upper()
    data['model_name'] = data['model_name'].str.replace(' ', '_')
    data['model_name'] = data['model_name'].str.replace('-', '_')
    data['model_name'] = data['model_name'].str.replace('KLASSE', 'CLASS')
    data['model_name'] = data['model_name'].str.replace('КЛАСС', 'CLASS')
    data['model_name'] = data['model_name'].str.replace('СЕРИИ', 'SERIES')
    data['model_name'] = data['model_name'].str.replace('RAV4', 'RAV_4')
    data['model_name'] = data['model_name'].str.replace('RS6', 'RS_6')
    data['model_name'] = data['model_name'].str.replace('RSQ3', 'RS_Q3')
    data['model_name'] = data['model_name'].str.replace('RS7', 'RS_7')
    data['model_name'] = data['model_name'].str.replace('RS4', 'RS_4')
    data['model_name'] = data['model_name'].str.replace('RS3', 'RS_3')
    data['model_name'] = data['model_name'].str.replace('RS5', 'RS_5')
    data['model_name'] = data['model_name'].str.replace('DELICA_D:5', 'DELICA_D5')
    data['model_name'] = data['model_name'].str.replace('DELICA_D_5', 'DELICA_D5')
    data['model_name'] = data['model_name'].str.replace('DELICA_D:2', 'DELICA_D2')
    data['model_name'] = data['model_name'].str.replace('QASHQAI\\+2', 'QASHQAI_PLUS_2')
    data['model_name'] = data['model_name'].str.replace('\\(NORTH_AMERICA\\)', 'NA')
    data['model_name'] = data['model_name'].str.replace("'", '')
    data['model_name'] = data['model_name'].str.replace(":", '')
    data['model_name'] = data['model_name'].str.replace('Z3M', 'Z3_M')
    data['model_name'] = data['model_name'].str.replace('Z3М', 'Z3_M')
    data['model_name'] = data['model_name'].str.replace('V40_CC', 'V40_CROSS_COUNTRY')
    data['model_name'] = data['model_name'].str.replace('^140$', '140_SERIES')
    data['model_name'] = data['model_name'].str.replace('S_CLASS_MAYBACH', 'MAYBACH_S_CLASS')
    data['model_name'] = data['model_name'].str.replace('GLE_CLASS_COUPE_AMG', 'GLE_COUPE_AMG')
    data['model_name'] = data['model_name'].str.replace('PRIUS_V_\(\+\)', 'PRIUS_PLUS')
    data['model_name'] = data['model_name'].str.replace('PRIUSPLUS', 'PRIUS_PLUS')
    data['model_name'] = data['model_name'].str.replace('MASTERACE_SURF', 'MASTER_ACE_SURF')
    data['model_name'] = data['model_name'].str.replace('STEPWGN', 'STEPWAGON')
    data['model_name'] = data['model_name'].str.replace('AMG_GLC_COUPE', 'GLC_COUPE_AMG')
    data['model_name'] = data['model_name'].str.replace('^GLS_AMG$', 'GLS_CLASS_AMG')
    data['model_name'] = data['model_name'].str.replace('^GLB_AMG$', 'GLB_CLASS_AMG')
    data['model_name'] = data['model_name'].str.replace('^GLA_AMG$', 'GLA_CLASS_AMG')
    data['model_name'] = data['model_name'].str.replace('^GLE_AMG$', 'GLE_CLASS_AMG')
    data['model_name'] = data['model_name'].str.replace('^GLC_AMG$', 'GLC_CLASS_AMG')
    data['model_name'] = data['model_name'].str.replace('^GLS_COUPE$', 'GLS_CLASS_COUPE')
    data['model_name'] = data['model_name'].str.replace('^GLA_COUPE$', 'GLA_CLASS_COUPE')
    data['model_name'] = data['model_name'].str.replace('^GLE_COUPE$', 'GLE_CLASS_COUPE')
    data['model_name'] = data['model_name'].str.replace('^GLC_COUPE$', 'GLC_CLASS_COUPE')
    data['model_name'] = data['model_name'].str.replace('^GLB_COUPE$', 'GLB_CLASS_COUPE')
    
    return data['model_name']

In [21]:
clean_model(all_cars)

0         OCTAVIA
1         OCTAVIA
2            YETI
3         OCTAVIA
4         OCTAVIA
           ...   
104656     GALANT
104658     PAJERO
104659     GALANT
104660     DELICA
104661     PAJERO
Name: model_name, Length: 94352, dtype: object

In [22]:
len(all_cars.model_name.unique())

632

In [23]:
all_cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94352 entries, 0 to 104661
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            94352 non-null  int64  
 1   bodyType              94352 non-null  object 
 2   brand                 94352 non-null  object 
 3   car_url               94352 non-null  object 
 4   color                 94352 non-null  object 
 5   description           94352 non-null  object 
 6   engineDisplacement    94352 non-null  object 
 7   enginePower           94352 non-null  object 
 8   equipment_dict        94352 non-null  object 
 9   fuelType              94352 non-null  object 
 10  mileage               94352 non-null  float64
 11  modelDate             94352 non-null  float64
 12  model_info            94352 non-null  object 
 13  model_name            94352 non-null  object 
 14  numberOfDoors         94352 non-null  float64
 15  productionDate    

сохраним этот датасет для дальнейшей работы и проведения EDA, Feature Engineering и посторения моделей

In [24]:
all_cars.to_csv("dataset_V2.csv")

то же проделаем с тестовым датасетом, извлечем из признака super_gen данные о расходе топлива

In [25]:
test = pd.read_csv('test.csv')
test.sample(3)

,Unnamed: 0,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,rate_fuel
18783,18783,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Внимание! Только для клиентов AVILON Автомобил...,6.0 LTR,530 N12,"{""cruise-control"":true,""esp"":true,""usb"":true,""...",...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,11.3
28997,28997,хэтчбек 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/cari...,серый,NaN,Машина достойная. Продаю не срочно!!!,1.6 LTR,103 N12,NaN,...,механическая,JAPANESE,3 или более,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,7.4
25362,25362,внедорожник 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/toua...,коричневый,NaN,Автомобиль в идеальном состоянии. Уникальный а...,3.6 LTR,249 N12,NaN,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,10.9


In [4]:
test['super_gen'][0]

'{"id":"10373605","displacement":1197,"engine_type":"GASOLINE","gear_type":"FORWARD_CONTROL","transmission":"ROBOT","power":105,"power_kvt":77,"human_name":"1.2 AMT (105 л.с.)","acceleration":10.5,"clearance_min":155,"fuel_rate":5}'

In [7]:
test['rate_split'] = test['super_gen'].str.split(',')
test['rate_split'][38]

['{"id":"3480870"',
 '"displacement":1781',
 '"engine_type":"GASOLINE"',
 '"gear_type":"FORWARD_CONTROL"',
 '"transmission":"AUTOMATIC"',
 '"power":150',
 '"power_kvt":110',
 '"human_name":"1.8 AT (150 л.с.)"',
 '"acceleration":10.9}']

In [9]:
# создадим список, куда могут попасть все данные о расходе топлива
rate = []
for d in test['rate_split']:
    rate.append(d[-2:])  
rate

[['"clearance_min":155', '"fuel_rate":5}'],
 ['"clearance_min":156', '"fuel_rate":6.4}'],
 ['"fuel_rate":7.1', '"clearance_max":158}'],
 ['"clearance_min":155', '"fuel_rate":6.7}'],
 ['"clearance_min":164', '"fuel_rate":7.7}'],
 ['"clearance_min":127', '"fuel_rate":7.7}'],
 ['"clearance_min":180', '"fuel_rate":8}'],
 ['"clearance_min":164', '"fuel_rate":7.4}'],
 ['"clearance_min":187', '"fuel_rate":6.8}'],
 ['"clearance_min":180', '"fuel_rate":8}'],
 ['"clearance_min":170', '"fuel_rate":5.8}'],
 ['"fuel_rate":4.9', '"clearance_max":156}'],
 ['"clearance_min":164', '"fuel_rate":5.8}'],
 ['"clearance_min":187', '"fuel_rate":7.1}'],
 ['"clearance_min":164', '"fuel_rate":7.9}'],
 ['"clearance_min":187', '"fuel_rate":6.1}'],
 ['"fuel_rate":5.8', '"clearance_max":156}'],
 ['"clearance_min":187', '"fuel_rate":6.1}'],
 ['"clearance_min":170', '"fuel_rate":6.1}'],
 ['"clearance_min":170', '"fuel_rate":6.1}'],
 ['"clearance_min":187', '"fuel_rate":7.1}'],
 ['"clearance_min":187', '"fuel_rate":5.

In [13]:
# извлечем из него только данные о расходе
rate2 = []
for fuel in rate:
    if "fuel_rate" in fuel[0]:
            rate2.append(fuel[0])
    elif "fuel_rate" in fuel[1]:
            rate2.append(fuel[1])
    elif "fuel_rate" not in fuel:
            rate2.append('w') 
            
test['rate'] = rate2
test['rate']

0           "fuel_rate":5}
1         "fuel_rate":6.4}
2          "fuel_rate":7.1
3         "fuel_rate":6.7}
4         "fuel_rate":7.7}
               ...        
34681     "fuel_rate":5.8}
34682     "fuel_rate":4.9}
34683    "fuel_rate":10.3}
34684     "fuel_rate":5.4}
34685     "fuel_rate":8.2}
Name: rate, Length: 34686, dtype: object

In [14]:
# "почистим" список и разложим его на 2 части
test['rate'] = test['rate'].str.replace(('}'), '')
test['rate'] = test['rate'].str.replace(('"'), '')
test['rate_split'] = test['rate'].str.split(':')

In [15]:
rate3 = []
for fuel in test['rate_split']:
    if len(fuel)> 1:
        rate3.append(fuel[1])
    else:
        rate3.append(fuel[0])
        
test['rate2'] = rate3
test['rate2'].value_counts()

w       3476
6.7      977
7.4      860
7.1      836
7.9      792
        ... 
15.2       1
0.7        1
3          1
3.7        1
2.5        1
Name: rate2, Length: 148, dtype: int64

In [17]:
# посчитаем среднее, моду и медиану не считая пропусков "w"
rate_test_w = test[~test.rate2.str.match("w", na=False)]
rate_test_w['rate2'] = pd.to_numeric(rate_test_w['rate2'])

<ipython-input-17-309da5a1009a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate_test_w['rate2'] = pd.to_numeric(rate_test_w['rate2'])


In [18]:
print (rate_test_w['rate2'].median())
print(rate_test_w['rate2'].mean())
print (rate_test_w['rate2'].mode())

7.8
8.263062800384338
0    6.7
dtype: float64


In [19]:
# заполним медианой пропуски и преобразуем данные в числовой формат
test['rate2'] = test['rate2'].str.replace(('w'), '7.8')
test['rate_fuel'] = pd.to_numeric(test['rate2'])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              34686 non-null  object 
 1   brand                 34686 non-null  object 
 2   car_url               34686 non-null  object 
 3   color                 34686 non-null  object 
 4   complectation_dict    6418 non-null   object 
 5   description           34686 non-null  object 
 6   engineDisplacement    34686 non-null  object 
 7   enginePower           34686 non-null  object 
 8   equipment_dict        24690 non-null  object 
 9   fuelType              34686 non-null  object 
 10  image                 34686 non-null  object 
 11  mileage               34686 non-null  int64  
 12  modelDate             34686 non-null  int64  
 13  model_info            34686 non-null  object 
 14  model_name            34686 non-null  object 
 15  name               

In [20]:
# удалим лишние столбцы и запишем изменения в исходный датасет
test.drop(['rate_split', 'rate', 'rate2'], axis=1, inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              34686 non-null  object 
 1   brand                 34686 non-null  object 
 2   car_url               34686 non-null  object 
 3   color                 34686 non-null  object 
 4   complectation_dict    6418 non-null   object 
 5   description           34686 non-null  object 
 6   engineDisplacement    34686 non-null  object 
 7   enginePower           34686 non-null  object 
 8   equipment_dict        24690 non-null  object 
 9   fuelType              34686 non-null  object 
 10  image                 34686 non-null  object 
 11  mileage               34686 non-null  int64  
 12  modelDate             34686 non-null  int64  
 13  model_info            34686 non-null  object 
 14  model_name            34686 non-null  object 
 15  name               

также очистим данные по моделям авто/приведем их к единому формату в тестовой базе

In [27]:
test.model_name.value_counts()

OCTAVIA            1418
5ER                1026
LANCER              855
E_KLASSE            809
PASSAT              782
                   ... 
W186                  1
FAIRLADY_Z            1
LANCER_RALLIART       1
SCEPTER_SEDAN         1
VANEO                 1
Name: model_name, Length: 544, dtype: int64

In [28]:
clean_model(test)

0        OCTAVIA
1        OCTAVIA
2         SUPERB
3        OCTAVIA
4        OCTAVIA
          ...   
34681        3ER
34682        5ER
34683        5ER
34684         X1
34685         X5
Name: model_name, Length: 34686, dtype: object

In [29]:
len(test.model_name.value_counts())

544

сохраним эти преобразования в исходный дататест для дальнейшоего анализа

In [30]:
test.to_csv("test.csv")